## Linear regression

In [45]:
require 'torch'
require 'nn'
require 'image'
require 'optim'
require 'csvigo'

In [46]:
-- reload the array
data = torch.Tensor{ csvigo.load{path='ex1data1.txt', mode='raw'} }

<csv>	parsing file: ex1data1.txt	
<csv>	parsing done	


In [47]:
data = data[1]

In [49]:
model = nn.Sequential()
ninputs = 1; noutputs = 1
model:add(nn.Linear(ninputs, noutputs))

  6.1101  17.5920
  5.5277   9.1302
  8.5186  13.6620
  7.0032  11.8540
  5.8598   6.8233
  8.3829  11.8860
  7.4764   4.3483
  8.5781  12.0000
  6.4862   6.5987
  5.0546   3.8166
  5.7107   3.2522
 14.1640  15.5050
  5.7340   3.1551
  8.4084   7.2258
  5.6407   0.7162
  5.3794   3.5129
  6.3654   5.3048
  5.1301   0.5608
  6.4296   3.6518
  7.0708   5.3893
  6.1891   3.1386
 20.2700  21.7670
  5.4901   4.2630
  6.3261   5.1875
  5.5649   3.0825
 18.9450  22.6380
 12.8280  13.5010
 10.9570   7.0467
 13.1760  14.6920
 22.2030  24.1470
  5.2524  -1.2200
  6.5894   5.9966
  9.2482  12.1340
  5.8918   1.8495
  8.2111   6.5426
  7.9334   4.5623
  8.0959   4.1164
  5.6063   3.3928
 12.8360  10.1170
  6.3534   5.4974
  5.4069   0.5566
  6.8825   3.9115
 11.7080   5.3854
  5.7737   2.4406
  7.8247   6.7318
  7.0931   1.0463
  5.0702   5.1337
  5.8014   1.8440
 11.7000   8.0043
  5.5416   1.0179
  7.5402   6.7504
  5.3077   1.8396
  7.4239   4.2885
  7.6031   4.9981
  6.3328   1.4233
  6.3589  

In [51]:
criterion = nn.MSECriterion()

In [52]:
x, dl_dx = model:getParameters()

In [53]:
feval = function()
   _nidx_ = (_nidx_ or 0) + 1
   if _nidx_ > (#data)[1] then _nidx_ = 1 end
   local sample = data[_nidx_]
   local target = sample[{ {2} }]      -- this funny looking syntax allows
   local inputs = sample[{ {1} }]    -- slicing of arrays.
   dl_dx:zero()
   local loss_x = criterion:forward(model:forward(data), target)
   model:backward(inputs, criterion:backward(model.output, target))
   return loss_x, dl_dx
end

In [54]:
sgd_params = {
   learningRate = 1e-4,
   learningRateDecay = 1e-5,
   weightDecay = 0,
   momentum = 0
}

In [55]:
for i = 1,1e4 do
   current_loss = 0
  _,fs = optim.sgd(feval,x,sgd_params)
  current_loss = fs[1]
   if i%100 == 0 then
        print('current loss = ' .. current_loss)
   end
end


/root/torch/install/share/lua/5.1/nn/Linear.lua:51: size mismatch, m1: [97 x 2], m2: [1 x 1] at /root/torch/pkg/torch/lib/TH/generic/THTensorMath.c:706
stack traceback:
	[C]: in function 'addmm'
	/root/torch/install/share/lua/5.1/nn/Linear.lua:51: in function 'updateOutput'
	/root/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function 'forward'
	[string "feval = function()..."]:8: in function 'opfunc'
	/root/torch/install/share/lua/5.1/optim/sgd.lua:44: in function 'sgd'
	[string "for i = 1,1e4 do..."]:3: in main chunk
	[C]: in function 'xpcall'
	/root/torch/install/share/lua/5.1/itorch/main.lua:179: in function </root/torch/install/share/lua/5.1/itorch/main.lua:143>
	/root/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/ro

In [56]:

myPrediction = model:forward(data[1][{ {1} }])

In [57]:
data[1][{{2}}][1]

17.592	


In [58]:
print('id  approx   text')
current_loss = 0
for i = 1,(#data)[1] do
   local myPrediction = model:forward(data[i][{ {1} }])
   print(string.format("%2d  %6.2f %6.2f", i, myPrediction[1], data[i][2]))
    current_loss = current_loss + (myPrediction[1] - data[i][{ {2} }][1])*(myPrediction[1] - data[i][{ {2} }][1])
end
current_loss = current_loss / (#data)[1]
print('evaluation = ' .. current_loss)

id  approx   text	
 1   -0.82  17.59	
 2   -0.81   9.13	
 3   -0.85  13.66	
 4   -0.83  11.85	
 5   -0.81   6.82	
 6   -0.85  11.89	
 7   -0.83   4.35	
 8   -0.85  12.00	
 9   -0.82   6.60	
10   -0.80   3.82	
11   -0.81   3.25	
12   -0.92  15.51	
13   -0.81   3.16	
14   -0.85   7.23	
15   -0.81   0.72	
16   -0.81   3.51	
17   -0.82   5.30	
18   -0.80   0.56	
19   -0.82   3.65	
20   -0.83   5.39	


21   -0.82   3.14	
22   -1.01  21.77	
23   -0.81   4.26	
24   -0.82   5.19	
25   -0.81   3.08	
26   -0.99  22.64	
27   -0.91  13.50	
28   -0.88   7.05	
29   -0.91  14.69	
30   -1.03  24.15	
31   -0.80  -1.22	
32   -0.82   6.00	
33   -0.86  12.13	
34   -0.81   1.85	
35   -0.84   6.54	
36   -0.84   4.56	
37   -0.84   4.12	
38   -0.81   3.39	
39   -0.91  10.12	
40   -0.82   5.50	
41   -0.81   0.56	
42   -0.83   3.91	
43   -0.89   5.39	
44   -0.81   2.44	


45   -0.84   6.73	
46   -0.83   1.05	
47   -0.80   5.13	
48   -0.81   1.84	
49   -0.89   8.00	
50   -0.81   1.02	
51   -0.84   6.75	
52   -0.80   1.84	
53   -0.83   4.29	
54   -0.84   5.00	
55   -0.82   1.42	
56   -0.82  -1.42	
57   -0.82   2.48	
58   -0.81   4.60	
59   -0.86   3.96	
60   -0.86   5.41	
61   -0.85   5.17	
62   -0.80  -0.74	
63   -1.02  17.93	
64   -0.93  12.05	
65   -0.99  17.05	
66   -0.83   4.89	


67   -0.85   5.74	
68   -0.87   7.78	
69   -0.81   1.02	
70   -1.01  20.99	
71   -0.87   6.68	
72   -0.83   4.03	
73   -0.81   1.28	
74   -0.83   3.34	
75   -0.80  -2.68	
76   -0.82   0.30	
77   -0.84   3.88	
78   -0.80   5.70	
79   -0.87   6.75	
80   -0.80   2.06	
81   -0.81   0.48	
82   -0.80   0.20	
83   -0.82   0.68	
84   -0.87   7.54	
85   -0.82   5.34	
86   -0.85   4.24	
87   -0.86   6.80	
88   -0.81   0.93	
89   -0.81   0.15	
90   -0.80   2.82	
91   -0.81   1.85	


92   -0.84   4.30	
93   -0.81   7.20	
94   -0.80   1.99	
95   -0.85   0.14	
96   -0.91   9.06	
97   -0.81   0.62	
evaluation = 75.187102156819	
